In [1]:
%pylab inline
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
import os
from itertools import product,combinations
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import StandardScaler,PolynomialFeatures

Populating the interactive namespace from numpy and matplotlib


In [ ]:
!conda install -c statsmodels statsmodels=0.8

In [4]:
path=r"C:\Reports\FC"
data = pd.read_csv(os.path.join(path,'total_sales.csv'),';', index_col=['month'], parse_dates=['month'], dayfirst=True)

fulldata=data
data=data.ix[:-3]
data.tail(3)

,sales
month,
2016-10-01,7768538609
2016-11-01,8179133401
2016-12-01,12140414460


In [ ]:
#обратное преобразование Бокса Кокса
def invboxcox(y,lmbda):
   if lmbda == 0:
      return(np.exp(y))
   else:
      return(np.exp(np.log(lmbda*y+1)/lmbda))

In [ ]:
plt.figure(figsize(15,7))
data.plot()
#plt.ylabel(u'')
#plt.title('')
pylab.show()

In [ ]:
plt.figure(figsize(15,10))
sm.tsa.seasonal_decompose(data.sales,'multiplicative').plot()
print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(data.sales)[1])

In [ ]:
pd.concat([sm.tsa.seasonal_decompose(data.sales,'multiplicative').trend,
           sm.tsa.seasonal_decompose(data.sales,'multiplicative').seasonal],axis=1).to_excel(
    os.path.join(path,'trend_season.xlsx'))

In [ ]:
data['sales'], lmbda = stats.boxcox(data.sales)
"Оптимальный параметр преобразования Бокса-Кокса: %f" % lmbda

In [ ]:
def data_diff(df,column,arr):
    k=1
    df['sales_diff'] = df[column] - df[column].shift(arr[0]) 
    
    for el in arr[1:]:
        df['sales_diff'] = df['sales_diff'] - df['sales_diff'].shift(el)
        k=k+el
    return df,k
        
data,k=data_diff(data,'sales',[1])
print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(data['sales_diff'].dropna())[1])

plt.figure(figsize(15,10))
sm.tsa.seasonal_decompose(data.sales_diff.dropna()).plot()
pylab.show()

In [ ]:
lags=data.shape[0]-k-1
plt.figure(figsize(15,8))
ax = plt.subplot(211)
sm.graphics.tsa.plot_acf(data.sales_diff.dropna().values.squeeze(), lags=lags, ax=ax)
for i in range(12, lags, 12):
    plt.axvline(i, -0.3, 1, c='r')
#x = plt.subplot(212)
#sm.graphics.tsa.plot_pacf(data.sales.dropna().values.squeeze(), lags=12, ax=ax)
pylab.show()
ax = plt.subplot(212)
sm.graphics.tsa.plot_pacf(data.sales_diff.dropna().values.squeeze(), lags=lags, ax=ax)
for i in range(12, lags, 12):
    plt.axvline(i, -0.3, 1, c='r')
pylab.show()

In [ ]:
ps = range(0, 11)
d=1 # тк 0 раз дифференциорал
qs = range(0, 11)
Ps = range(0, 3)
D=0 # одно сезонное дифференцирование
Qs = range(0,3)
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [ ]:
[method for method in dir(sm.tsa) if callable(getattr(sm.tsa, method))]

In [ ]:
%%time
results = []
best_aic = float("inf")
warnings.filterwarnings('ignore')
column='sales'
for param in parameters_list:
    #try except нужен, потому что на некоторых наборах параметров модель не обучается
    try:
        model=SARIMAX(data[column], order=(param[0], d, param[1]), 
                                        seasonal_order=(param[2], D, param[3], 12)).fit(disp=-1)
    #выводим параметры, на которых модель не обучается и переходим к следующему набору
    except ValueError:
        print('wrong parameters:', param)
        continue
    aic = model.aic
    #сохраняем лучшую модель, aic, параметры
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results.append([param, model.aic,model])
    
warnings.filterwarnings('default')

In [ ]:
result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'aic','model']
result_table=result_table.sort_values(by = 'aic', ascending=True).head().reset_index(drop=True)
result_table

In [ ]:
#Лучшая модель:
print(best_model.summary())

In [ ]:
plt.figure(figsize(15,8))
plt.subplot(211)
best_model.resid.dropna().plot()
plt.ylabel(u'Residuals')

ax = plt.subplot(212)
sm.graphics.tsa.plot_acf(best_model.resid.dropna().values.squeeze(), lags=48, ax=ax)

print("Критерий Стьюдента: p=%f" % stats.ttest_1samp(best_model.resid.dropna(), 0)[1])
print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(best_model.resid.dropna())[1])

In [ ]:

q_test = sm.tsa.stattools.acf(model.resid.dropna(), qstat=True) #свойство resid, хранит остатки модели, qstat=True, означает что применяем указынный тест к коэф-ам
pd.DataFrame({'Q-stat':q_test[1], 'p-value':q_test[2]})
#из чего видим что остатки случайны

In [ ]:
#data['model'] = invboxcox(best_model.fittedvalues, lmbda)
data['model'] = result_table['model'][1].fittedvalues.shift(-1)
plt.figure(figsize(15,7))
data.sales.plot()
data.model.dropna().plot(color='r')
plt.ylabel('sales')
pylab.show()

In [ ]:
data2 = fulldata
date_list = [max(data2.index) + relativedelta(months=x) for x in range(0,12)]
future = pd.DataFrame(index=date_list, columns= data2.columns)
data2 = pd.concat([data2, future])
data2['forecast'] = best_model.predict(start=data.shape[0], end=data2.shape[0]).shift(-1)

In [ ]:
plt.figure(figsize(15,7))
data2.sales.dropna().plot()
data2.forecast.dropna().plot(color='r')
data.model.dropna().plot(color='r')
plt.ylabel('Wine sales')
pylab.show()

# Regression forecast

In [5]:
data = pd.read_csv(os.path.join(path,'total_sales_day.csv'),';', index_col=['day'], parse_dates=['day'], dayfirst=True)
data=data.ix[:,:3]

In [6]:
%%time
min_date=min(data.index)
data['day_of_week']=data.index.dayofweek
data['month']=data.index.month
data['total_days']=(data.index-min_date).days
data['num_day']=data.index.day
data['month_part']=data.num_day.apply(lambda x:x//8 )
data['week_num']=data.index.week

moving_week_window=3
moving_month_window=3

for i in range(1,moving_week_window+3):
    data[str(i)+'_week_before']=data.index-timedelta(days=7*i)
data['date_withot_day']=[data.index[i]-timedelta(days=int(data.num_day.values[i]-1))  for i in range(data.shape[0])]
for i in range(1,moving_month_window+3):
    data[str(i)+'_month_before']=data['date_withot_day'].apply(lambda x:x-relativedelta(months=+1*i))

Wall time: 37.8 s


In [7]:
data.tail(3)

,filial,store,sales,day_of_week,month,total_days,num_day,month_part,week_num,1_week_before,2_week_before,3_week_before,4_week_before,5_week_before,date_withot_day,1_month_before,2_month_before,3_month_before,4_month_before,5_month_before
day,,,,,,,,,,,,,,,,,,,,
2017-03-24,Филиал Южный,1162,2156235.37,4,3,1178,24,3,12,2017-03-17,2017-03-10,2017-03-03,2017-02-24,2017-02-17,2017-03-01,2017-02-01,2017-01-01,2016-12-01,2016-11-01,2016-10-01
2017-03-25,Филиал Южный,1162,2344565.01,5,3,1179,25,3,12,2017-03-18,2017-03-11,2017-03-04,2017-02-25,2017-02-18,2017-03-01,2017-02-01,2017-01-01,2016-12-01,2016-11-01,2016-10-01
2017-03-26,Филиал Южный,1162,2035948.87,6,3,1180,26,3,12,2017-03-19,2017-03-12,2017-03-05,2017-02-26,2017-02-19,2017-03-01,2017-02-01,2017-01-01,2016-12-01,2016-11-01,2016-10-01


In [8]:
%%time
def find_in_agg_df(df,pattern):
    try:r=df.ix[tuple(pattern)]['sales']
    except:r=np.nan
    return r

agg_month=data[['date_withot_day','sales','store']].pivot_table(index=['date_withot_day','store'],aggfunc=np.sum)
for i in range(1,moving_month_window+3):
    column_name=str(i)+'_month_before'
    data[column_name]=data[[column_name,'store']].apply(lambda row:find_in_agg_df(agg_month,row),axis=1)

Wall time: 5min 7s


In [9]:
%%time
def create_moving_week(df):
    df.reset_index(inplace=True)
    start_date=min_date+timedelta(days=6)
    res=df[df['day']>start_date][['day','store','1_week_before']].reset_index(drop=True)

    
    res['sales']=res.apply(lambda row:np.sum(df[(df['store']==row['store'])&
                                         (df['day']>=row['1_week_before'])&
                                         (df['day']<row['day'])]['sales']),axis=1)
    
    res.drop('1_week_before',axis=1,inplace=True)
    res.set_index(['day', 'store'],inplace=True)
    return res
agg_week_df=create_moving_week(data)

Wall time: 7min 17s


In [10]:
%%time
for i in range(1,moving_week_window+3):
    column_name=str(i)+'_week_before'
    data[column_name]=data[[column_name,'store']].apply(lambda row:find_in_agg_df(agg_week_df,row),axis=1)

Wall time: 5min 23s


In [11]:
data.dropna(inplace=True)
data.reset_index(inplace=True,drop=True)

In [12]:
for i in range(1,moving_week_window+1):
    
    column_names=[str(x)+'_week_before' for x in range(i,i+3) ]
    column_name_new=str(i)+'avg_week'
    data[column_name_new]=data[column_names].apply(np.mean,axis=1)
for i in range(1,moving_month_window+1):
    column_names=[str(x)+'_month_before' for x in range(i,i+3) ]
    column_name_new=str(i)+'avg_month'
    data[column_name_new]=data[column_names].apply(np.mean,axis=1)

columns=[str(x)+'_week_before' for x in range(1,moving_week_window+3)]+[
    str(x)+'_month_before' for x in range(1,moving_month_window+3)]
data.drop(columns,axis=1,inplace=True)

In [ ]:
data.columns

In [13]:
def generate_features(df,columns,degree,suffix):
    polynom_arr=PolynomialFeatures(degree).fit_transform(df[columns])
    polynom_df=pd.DataFrame(polynom_arr,columns=[str(el )+suffix for el in range(polynom_arr.shape[1])])
    
    for comb in combinations(range(len(columns)),2):
    
        polynom_df[suffix+columns[comb[0]]+'+'+columns[comb[1]]]=df[columns[comb[0]]]+df[columns[comb[1]]]

        polynom_df[suffix+columns[comb[0]]+'-'+columns[comb[1]]]=df[columns[comb[0]]]-df[columns[comb[1]]]
        polynom_df[suffix+columns[comb[0]]+'/'+columns[comb[1]]]=df[columns[comb[0]]]/df[columns[comb[1]]]
        polynom_df[suffix+columns[comb[0]]+'*'+columns[comb[1]]]=df[columns[comb[0]]]*df[columns[comb[1]]]
        
    return polynom_df
#generate_features(data,avg_week_cols,2,'month').head()    

In [14]:
avg_month_cols=[str(i)+'avg_month' for i in range(1,moving_month_window+1)]
avg_week_cols=[str(i)+'avg_week' for i in range(1,moving_week_window+1)]

multiple_df=pd.concat([generate_features(data,avg_month_cols,2,'month'),
                generate_features(data,avg_week_cols,2,'week')],axis=1)

In [21]:
avg_month_cols=[str(i)+'avg_month' for i in range(1,moving_month_window+1)]
avg_week_cols=[str(i)+'avg_week' for i in range(1,moving_week_window+1)]

real_features=['sales','total_days']+list(multiple_df.columns.values)#+avg_month_cols+avg_week_cols
cat_features=['filial','store','day_of_week', 'month', 'month_part','week_num']

In [22]:
data=pd.concat([data,multiple_df],axis=1)
data.drop(avg_month_cols+avg_week_cols,inplace=True,axis=1)

In [23]:
def transform(df,real_columns,cat_columns):
    df.is_a_copy=False

    df_real=df[real_columns]
    scaler = StandardScaler()
    df_real=pd.DataFrame(scaler.fit_transform(df_real),columns=real_columns)
    res=[df_real]
    df_cat=df[cat_columns]
    for column in df_cat.columns.values:
        data_slice=df_cat[column].astype(str)
        res.append(pd.get_dummies(data_slice,prefix =column, dummy_na=False))        
    res=pd.concat(res,axis=1)
    return res

transform(data,real_features,cat_features)

,sales,total_days,0month,1month,2month,3month,4month,5month,6month,7month,...,week_num_49,week_num_5,week_num_50,week_num_51,week_num_52,week_num_53,week_num_6,week_num_7,week_num_8,week_num_9
0,-0.476753,-1.750395,0.0,-0.448309,-0.468976,-0.489698,-0.457361,-0.464232,-0.472182,-0.469009,...,0,0,0,0,0,0,0,0,0,0
1,-0.586058,-1.747052,0.0,-0.448309,-0.468976,-0.489698,-0.457361,-0.464232,-0.472182,-0.469009,...,0,0,0,0,0,0,0,0,0,0
2,-0.587475,-1.743710,0.0,-0.448309,-0.468976,-0.489698,-0.457361,-0.464232,-0.472182,-0.469009,...,0,0,0,0,0,0,0,0,0,0
3,-0.570020,-1.740367,0.0,-0.448309,-0.468976,-0.489698,-0.457361,-0.464232,-0.472182,-0.469009,...,0,0,0,0,0,0,0,0,0,0
4,-0.471132,-1.737025,0.0,-0.448309,-0.468976,-0.489698,-0.457361,-0.464232,-0.472182,-0.469009,...,0,0,0,0,0,0,0,0,0,0
5,-0.131851,-1.733682,0.0,-0.448309,-0.468976,-0.489698,-0.457361,-0.464232,-0.472182,-0.469009,...,0,0,0,0,0,0,0,0,0,0
6,-0.237677,-1.730340,0.0,-0.448309,-0.468976,-0.489698,-0.457361,-0.464232,-0.472182,-0.469009,...,0,0,0,0,0,0,0,0,0,0
7,-0.489535,-1.726997,0.0,-0.448309,-0.468976,-0.489698,-0.457361,-0.464232,-0.472182,-0.469009,...,0,0,0,0,0,0,0,0,0,0
8,-0.542823,-1.723655,0.0,-0.448309,-0.468976,-0.489698,-0.457361,-0.464232,-0.472182,-0.469009,...,0,0,0,0,0,0,0,0,0,0
9,-0.403132,-1.720312,0.0,-0.448309,-0.468976,-0.489698,-0.457361,-0.464232,-0.472182,-0.469009,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
plt.figure(figsize(15,7))
data.sales.plot()
#plt.ylabel(u'')
#plt.title('')
pylab.show()